In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import * 

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Column 제거

In [2]:
train_df = pd.read_csv('../data/train.csv')
train_df.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,0,1,106,지방도1112호선,0,0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,0,2,103,일반국도11호선,0,0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,0,2,103,일반국도16호선,0,0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,0,2,107,태평로,0,0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,0,2,103,일반국도12호선,0,0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [3]:
train_df.columns

Index(['id', 'base_date', 'day_of_week', 'base_hour', 'road_in_use',
       'lane_count', 'road_rating', 'road_name', 'multi_linked',
       'connect_code', 'maximum_speed_limit', 'vehicle_restricted',
       'weight_restricted', 'height_restricted', 'road_type',
       'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted', 'target'],
      dtype='object')

In [4]:
for i in train_df.columns:
    print(len(train_df[i].unique()))

4701217
281
7
24
2
3
3
61
2
2
6
1
4
1
2
487
586
586
2
487
586
586
2
102


In [5]:
test_df = pd.read_csv('../data/test.csv')
test_df.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,목,17,0,3,107,연삼로,0,0,...,0.0,0,산지2교,33.499427,126.541298,없음,제주은행사거리,33.500772,126.543837,있음
1,TEST_000001,20220809,화,12,0,2,103,일반국도12호선,0,0,...,0.0,3,중문입구,33.258507,126.427003,없음,관광단지입구,33.258119,126.415840,없음
2,TEST_000002,20220805,금,2,0,1,103,일반국도16호선,0,0,...,0.0,0,도순3교,33.258960,126.476508,없음,도순2교,33.259206,126.474687,없음
3,TEST_000003,20220818,목,23,0,3,103,일반국도11호선,0,0,...,0.0,0,아라주공아파트,33.473494,126.545647,없음,인다마을,33.471061,126.545467,없음
4,TEST_000004,20220810,수,17,0,3,106,번영로,0,0,...,0.0,0,부록교 시종점,33.501477,126.569223,없음,봉개교 시종점,33.496863,126.581230,없음


In [6]:
train_df.columns

Index(['id', 'base_date', 'day_of_week', 'base_hour', 'road_in_use',
       'lane_count', 'road_rating', 'road_name', 'multi_linked',
       'connect_code', 'maximum_speed_limit', 'vehicle_restricted',
       'weight_restricted', 'height_restricted', 'road_type',
       'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted', 'target'],
      dtype='object')

In [7]:
drop_col = ['id', 'vehicle_restricted', 'height_restricted','start_node_name', 'start_longitude', 
            'end_node_name', 'end_longitude', 'base_date', 'road_in_use']
x = train_df.drop(drop_col, axis = 1)
x = x.drop('target', axis = 1)
y = train_df.loc[:, 'target']

test_data = test_df.drop(drop_col, axis = 1)

In [8]:
dumm_cols = ['day_of_week', 'base_hour', 'road_rating', 'road_name', 
             'multi_linked', 'connect_code', 'road_type', 
             'start_turn_restricted', 'end_turn_restricted']

In [9]:
for col in dumm_cols:
    ohe = OneHotEncoder(sparse=False) # 나중에 drop 적용해보기
    
    temp_x = ohe.fit_transform(x[col].values.reshape(-1,1))
#     temp_x_df = pd.DataFrame(temp_x, columns=ohe.categories_[0])
    temp_x_df = pd.DataFrame(temp_x, columns=ohe.get_feature_names([col]))
    x.drop(col, axis=1, inplace = True)
    x = pd.concat([x, temp_x_df], axis = 1)
    
    temp_test = ohe.transform(test_data[col].values.reshape(-1,1))
#     temp_test_df = pd.DataFrame(temp_test, columns=ohe.categories_[0])
    temp_test_df = pd.DataFrame(temp_test, columns=ohe.get_feature_names([col]))
    test_data.drop(col, axis=1, inplace = True)
    test_data = pd.concat([test_data, temp_test_df], axis = 1)

In [10]:
# for col in dumm_cols:
#     le = LabelEncoder()
#     x[col] = le.fit_transform(x[col])
#     test_data[col] = le.transform(x_test[col])

In [11]:
x.shape

(4701217, 110)

In [12]:
test_data.shape

(291241, 110)

In [18]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 2022)

In [14]:
# x_train.to_csv('../data/train1_origin/x_train.csv', index=False)
# x_val.to_csv('../data/train1_origin/x_val.csv', index=False)
# y_train.to_csv('../data/train1_origin/y_train.csv', index=False)
# y_val.to_csv('../data/train1_origin/y_val.csv', index=False)
# test_data.to_csv('../data/train1_origin/x_test.csv', index=False)

In [12]:
from sklearn.model_selection import GridSearchCV
 
model = RandomForestRegressor(n_estimators = 5, max_depth = 3, verbose = 0)
 
# grid_search = GridSearchCV(forest_reg, param_grid, cv=5, verbose = 10)
 
model.fit(x, y)

RandomForestRegressor(max_depth=3, n_estimators=5)

In [13]:
pred = model.predict(test_data)

In [14]:
result = pd.DataFrame({'id':test_df['id'], 'target':pred})
result.head()

,id,target
0,TEST_000000,28.693139
1,TEST_000001,46.297487
2,TEST_000002,46.297487
3,TEST_000003,35.713850
4,TEST_000004,35.713850


In [15]:
result.to_csv('../data/submission_v2.csv', index=False)

In [17]:
from pycaret.regression import *

In [ ]:
setup(session_id = 2022, data = pd.concat([x_train,y_train],axis=1), target = 'target',
      test_data = pd.concat([x_val,y_val],axis=1),
      normalize = True, normalize_method = 'zscore',
      transformation=True, 
      #fold_strategy='stratifiedkfold', 
      use_gpu = True)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:27:52
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


In [38]:
x_train.corr()

KeyboardInterrupt: 

In [11]:
for i in x:
    for j in x:
        if i == j:
            print(j)

lane_count
maximum_speed_limit
weight_restricted
start_latitude
end_latitude
day_of_week_금
day_of_week_목
day_of_week_수
day_of_week_월
day_of_week_일
day_of_week_토
day_of_week_화
base_hour_0
base_hour_1
base_hour_2
base_hour_3
base_hour_4
base_hour_5
base_hour_6
base_hour_7
base_hour_8
base_hour_9
base_hour_10
base_hour_11
base_hour_12
base_hour_13
base_hour_14
base_hour_15
base_hour_16
base_hour_17
base_hour_18
base_hour_19
base_hour_20
base_hour_21
base_hour_22
base_hour_23
road_rating_103
road_rating_106
road_rating_107
road_name_-
road_name_경찰로
road_name_고평교
road_name_관광단지1로
road_name_관광단지2로
road_name_관광단지로
road_name_관덕로
road_name_권학로
road_name_남조로
road_name_동문로
road_name_동부관광도로
road_name_동홍로
road_name_번영로
road_name_산서로
road_name_삼무로
road_name_삼봉로
road_name_삼성로
road_name_새서귀로
road_name_서사로
road_name_수영장길
road_name_시민광장로
road_name_시청로
road_name_신광로
road_name_신대로
road_name_신산로
road_name_아봉로
road_name_애원로
road_name_애조로
road_name_어시천교
road_name_연동로
road_name_연북2교
road_name_연북로
road_name_연삼